In [1]:
import datetime
import time

import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

from src.huge_dataset_project.InputLoader import InputFile
from src.huge_dataset_project.PickLoader import PickFile
from utils.data_reading.sound_data.station import StationsCatalog
from utils.physics.sound.sound_model import HomogeneousSoundModel
from utils.physics.sound.sound_velocity_grid import MonthlySoundVelocityGrid, MonthlySoundVelocityGridOptimized

/home/plerolland/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
picks = PickFile("../../huge_dataset_project/swarm2.pick")
input = InputFile("../../huge_dataset_project/Swarm2_data.input")
stations = StationsCatalog("/home/plerolland/Bureau/dataset.yaml").filter_out_undated().filter_out_unlocated()

acoustic_grid_interp = MonthlySoundVelocityGrid([f"../../../data/sound_model/min-velocities_month-{i:02d}.nc" for i in range(1,13)], interpolate=True)  # 3001.15s, mean error 0.030676462
acoustic_grid_interp_optimized = MonthlySoundVelocityGridOptimized([f"../../../data/sound_model/min-velocities_month-{i:02d}.nc" for i in range(1,13)], interpolate=True)  # 1507.42s, mean error 0.030739762
# with x0.997 : mean error 0.009412665
# with x0.996 : mean error 0.00859406
# with x0.995 : mean error 0.011902887
# with x0.993 : mean error 0.024033984
# with x0.992 : mean error 0.03129123
# with x0.99 : mean error 0.046128847
acoustic_grid = MonthlySoundVelocityGrid([f"../../../data/sound_model/min-velocities_month-{i:02d}.nc" for i in range(1,13)])  # 902.45s, mean error 0.043251827
acoustic_grid_optimized = MonthlySoundVelocityGridOptimized([f"../../../data/sound_model/min-velocities_month-{i:02d}.nc" for i in range(1,13)])  # 361.23 s, mean error 0.043365657
acoustic_homogeneous = HomogeneousSoundModel(sound_speed=1480) # 214.73s, mean error 0.05188065

In [3]:
acoustic_grid.get_sound_travel_time([0,0], [0,20], datetime.datetime(2015,3,1))

1501.7492066249183

In [7]:
models = [acoustic_grid_interp_optimized]

errors = [[] for _ in models]
x_list = [[] for _ in models]
for mi, model in enumerate(models):
    print(f'Evaluating model {mi}')
    t0 = time.time()
    for pick in tqdm(picks.picks):
        h_letters = pick.hydrophones
        h_names = [input.hydrophone_names[list(input.hydrophone_letters).index(h_letter)] for h_letter in h_letters]
        h_names = [h_name+"-bot" if h_name=="SWAMS" else h_name for h_name in h_names]
        
        available_s = [stations.by_date(pick.detection_times[i]).by_names(h_names[i])[0] for i in range(len(h_names))]
        
        positions = [s.get_pos() for s in available_s]
        p0 = pick.est_source_pos
        res = model.localize_common_source(positions, pick.detection_times, initial_pos=p0)
        est_pos = res.x[1:]
        est_time = np.min(pick.detection_times) + datetime.timedelta(seconds=res.x[0])
        errors[mi].append((est_pos[0]-pick.est_source_pos[0], est_pos[1]-pick.est_source_pos[1], est_time-pick.est_source_time))
        x_list[mi].append(res)
    t1 = time.time()
    print(f'Runtime of model {mi} is {t1-t0:.2f} s')
    
errors = np.array(errors)

Evaluating model 0


  0%|          | 19/4927 [00:04<19:58,  4.10it/s]

KeyboardInterrupt



In [5]:
to_plot = np.sqrt(np.square(errors[0,:,0].astype(np.float32)) + np.square(errors[0,:,1].astype(np.float32)))
print(np.mean(to_plot))

to_plot = np.delete(to_plot, np.abs(to_plot) > np.quantile(np.abs(to_plot), 0.95))
plt.hist(to_plot, bins=10)

KeyboardInterrupt: 

In [ ]:
x = np.array([np.mean(x.fun) for x in x_list[0]])
plt.plot(x)